In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.classification import *
from script import *

%reload_ext watermark
%watermark -iv -v -p pycaret

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.25.0

pycaret: 3.3.2

pandas    : 2.1.4
seaborn   : 0.13.2
matplotlib: 3.7.5
numpy     : 1.26.4



In [16]:
df = pd.read_csv('.\input\matches.csv',parse_dates='Date')
df

,Unnamed: 0,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,...,Match Report,NaN,13.0,1.0,17.8,0.0,0,0,2024,Liverpool
1,1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,...,Match Report,NaN,25.0,9.0,16.8,1.0,0,1,2024,Liverpool
2,2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,...,Match Report,NaN,9.0,4.0,17.2,1.0,0,0,2024,Liverpool
3,3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,...,Match Report,NaN,17.0,4.0,14.7,0.0,0,0,2024,Liverpool
4,4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,...,Match Report,NaN,16.0,5.0,15.8,0.0,0,0,2024,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,20,2023-12-30,15:00,Premier League,Matchweek 20,Sat,Away,L,0.0,2.0,...,Match Report,NaN,4.0,2.0,14.9,0.0,0,0,2024,SheffieldUnited
472,22,2024-01-21,14:00,Premier League,Matchweek 21,Sun,Home,D,2.0,2.0,...,Match Report,NaN,20.0,5.0,18.5,1.0,1,1,2024,SheffieldUnited
473,24,2024-01-30,20:00,Premier League,Matchweek 22,Tue,Away,L,2.0,3.0,...,Match Report,NaN,9.0,5.0,16.5,0.0,0,0,2024,SheffieldUnited
474,25,2024-02-03,17:30,Premier League,Matchweek 23,Sat,Home,L,0.0,5.0,...,Match Report,NaN,10.0,4.0,10.0,0.0,0,0,2024,SheffieldUnited


# Projeto Semantix

O objetivo deste projeto é encontrar uma problemática da vida real que possa ser solucionada através de análise de dados e machine learning. A ideia do projeto é explanar e **justificar** a **relevância** do uso de dados para encontrar a solução.

O tema em que vamos focar neste projeto é futebol, vamos baixar um dataset fornecido pelo [Kaggle](http://www.kaggle.com/) contendo todas as informações de cada partidade de futebol da Premier League do ano de 2023/24.

O objetivo será treinar um modelo de machine learning para prever qual time foi o vencedor ou se foi empate. Para isso iremos utilizar as bibliotecas, como o pandas, para nossa análise exploratória de dados e bibliotecas para a plotagem dos gráficos, como matplotlib e seaborn.

Para a modelagem iremos utilizar o PyCaret para buscar o melhor resultado possível.

 >O link para o dataset que se encontra no Kaggle é este: [LINK](https://www.kaggle.com/datasets/mertbayraktar/english-premier-league-matches-20232024-season)

---

## **1. Dados do Dataset:** 

Os dados representam informações dos jogos da Premier League do ano de 2023/24 e contam com as seguintes colunas: 

| *Coluna*            | *Explicação*                          |
|---------------------|---------------------------------------|
| Date                | Data                                  |
| Time                | Horário                               |
| Comp                | "Premier League"                      |
| Round               | Rodada da partida                     |
| Day                 | Dia da Semana                         |
| Venue               | "Home"                                |
| Result              | Resultado da partida ("W", "L" ou "D")|
| GF                  | Gols Marcados                         |
| GA                  | Gols Sofridos                         |
| Opponent            | Oponente                              |
| xG                  | Gols Marcados esperados               |
| xGA                 | Gols Sofridos esperados               |
| Poss                | Posse de bola                         |
| Attendance          | Público presente                      |
| Captain             | Capitão do Time                       |
| Formation           | Formação do Time                      |
| Referee             | Árbitro                               |
| Sh                  | Total de Chutes                       |
| SoT                 | Total de Chutes a Gol                 |
| Dist                | Distância Média  dos Chutes           |
| FK                  | Chutes de Falta                       |
| PK                  | Penaltis Convertidos                  |
| PKatt               | Penaltis Cobrados                     |
| Season              | "2024"                                |
| Team                | Time Mandante                         |
| Captain_Opponent    | Capitão do Time                       |
| Formation_Opponent  | Formação do Time                      |
| Sh_Opponent         | Total de Chutes                       |
| SoT_Opponent        | Total de Chutes a Gol                 |
| Dist_Opponent       | Distância Média dos Chutes            |
| FK_Opponent         | Chutes de Falta                       |
| PK_Opponent         | Penaltis Convertidos                  |
| PKatt_Opponent      | Penaltis Cobrados                     |


---

## **2. Análise Exploratória de Dados (EDA):** 

Vamos carregar nosso dataset e analisar cada um dos dados.

In [14]:
df.head()

NameError: name 'df' is not defined